## 1️⃣ Setup & Mount Drive

In [ ]:
# Clone repository
!git clone https://github.com/dungdinhhaha/AIDetect.git /content/ComparisonDetector 2>/dev/null || echo 'Repo already cloned'
%cd /content/ComparisonDetector

In [ ]:
# Install dependencies
!pip install -q tensorflow==2.19.0 matplotlib seaborn scikit-learn

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted!")

In [ ]:
# Check GPU
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
print(f"GPU available: {len(gpus) > 0}")
if gpus:
    print(f"GPU devices: {[gpu.name for gpu in gpus]}")
else:
    print("⚠️  No GPU detected! Go to Runtime > Change runtime type > Select GPU")

## 2️⃣ Verify Data Pipeline

In [ ]:
import os
from pathlib import Path

# Check if data exists
data_dir = Path('/content/drive/MyDrive/content/data/tct')
if data_dir.exists():
    files = list(data_dir.glob('*.tfrecord'))
    print(f"✅ Found {len(files)} TFRecord files:")
    for f in files:
        size_gb = f.stat().st_size / (1024**3)
        print(f"   - {f.name}: {size_gb:.2f} GB")
else:
    print(f"❌ Data directory not found: {data_dir}")
    print("Make sure to upload data to Google Drive first!")

In [ ]:
# Test data loading
from configs.config_v2 import IMAGE_SIZE, NUM_CLASSES, BATCH_SIZE
from data.loader_tf2 import build_dataset

print("Testing data pipeline...")
print(f"Image size: {IMAGE_SIZE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Num classes: {NUM_CLASSES}")

test_paths = ['/content/drive/MyDrive/content/data/tct/train.tfrecord']
ds = build_dataset(test_paths, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, is_training=False)

for images, targets in ds.take(1):
    print(f"\n✅ Data pipeline working!")
    print(f"   Images shape: {images.shape}")
    print(f"   Labels shape: {targets['labels'].shape}")
    print(f"   Boxes shape: {targets['boxes'].shape}")

## 3️⃣ 🚀 Start Training (Fixed Version)

**Key improvements:**
- `.repeat()` added - dataset won't run out of data
- Dynamic `steps_per_epoch` - calculated from actual data size
- Better callbacks - saves best model, reduces LR on plateau
- Informative logging - shows training progress clearly

In [ ]:
# Run training
print("🚀 Starting training...")
print("Expected duration: ~4-6 hours for 20 epochs on Tesla T4\n")

!python train_keras.py

## 4️⃣ Monitor Training (TensorBoard)

In [ ]:
# Load TensorBoard extension
%load_ext tensorboard

# Display TensorBoard
log_dir = '/content/drive/MyDrive/comparison_detector_models_v2/logs'
%tensorboard --logdir {log_dir}

## 5️⃣ Check Checkpoints

In [ ]:
import os
from pathlib import Path

ckpt_dir = Path('/content/drive/MyDrive/comparison_detector_models_v2/checkpoints')

if ckpt_dir.exists():
    ckpts = sorted(list(ckpt_dir.glob('ckpt_*.weights.h5')), key=lambda x: int(x.stem.split('_')[1]))
    print(f"✅ Found {len(ckpts)} checkpoints:")
    for ckpt in ckpts[-5:]:  # Show last 5
        size_mb = ckpt.stat().st_size / (1024*1024)
        print(f"   - {ckpt.name}: {size_mb:.1f} MB")
else:
    print("No checkpoints found yet. Training may still be in progress.")

## 6️⃣ Resume Training (if interrupted)

In [ ]:
# If training was interrupted, run this to resume
print("Resuming training from checkpoint...")
!python resume_training.py

## 7️⃣ Post-Training: Save Models

In [ ]:
import os
import json
import shutil
from pathlib import Path
from datetime import datetime

model_dir = Path('/content/drive/MyDrive/comparison_detector_models_v2')

print("📦 Post-Training: Saving Models & Archives")
print("=" * 60)

# Create archive folder
timestamp = datetime.now().strftime('%Y_%m_%d_%H%M%S')
archive_dir = model_dir / f'archive_{timestamp}'
archive_dir.mkdir(exist_ok=True)

# Copy best model
best_model_src = model_dir / 'best_model.h5'
if best_model_src.exists():
    shutil.copy(best_model_src, archive_dir / 'best_model.h5')
    size_mb = best_model_src.stat().st_size / (1024*1024)
    print(f"✅ Best model archived: {size_mb:.1f} MB")

# Copy final model
final_model_src = model_dir / 'final_model.keras'
if final_model_src.exists():
    shutil.copy(final_model_src, archive_dir / 'final_model.keras')
    size_mb = final_model_src.stat().st_size / (1024*1024)
    print(f"✅ Final model archived: {size_mb:.1f} MB")

# Create metadata
metadata = {
    'timestamp': timestamp,
    'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'framework': 'TensorFlow',
    'tf_version': '2.19.0',
    'epochs': 20,
    'batch_size': 2,
    'image_size': [640, 640],
    'num_classes': 12,
    'backbone': 'resnet50'
}

with open(archive_dir / 'metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\n✅ Archive saved to: {archive_dir}")
print("=" * 60)

## 8️⃣ Export Models for Deployment

In [ ]:
import tensorflow as tf
from pathlib import Path

model_dir = Path('/content/drive/MyDrive/comparison_detector_models_v2')
best_model_path = model_dir / 'best_model.h5'

if best_model_path.exists():
    print("📤 Exporting models...\n")
    
    # Load model
    model = tf.keras.models.load_model(str(best_model_path))
    
    # 1. TensorFlow SavedModel (for FastAPI/TF Serving)
    saved_model_dir = model_dir / 'saved_model'
    model.save(str(saved_model_dir), save_format='tf')
    print(f"✅ SavedModel exported to: saved_model/")
    
    # 2. TF Lite (for mobile/edge devices - optional)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()
    
    tflite_path = model_dir / 'model_quantized.tflite'
    with open(tflite_path, 'wb') as f:
        f.write(tflite_model)
    print(f"✅ TFLite model exported: model_quantized.tflite")
    
    # Show file sizes
    print("\n📊 Model sizes:")
    for name, path in [
        ('Best Model (H5)', model_dir / 'best_model.h5'),
        ('Final Model (Keras)', model_dir / 'final_model.keras'),
        ('TFLite Quantized', tflite_path)
    ]:
        if path.exists():
            size_mb = path.stat().st_size / (1024*1024)
            print(f"  {name}: {size_mb:.1f} MB")
else:
    print("❌ Best model not found. Training may not be complete.")

## 9️⃣ Next Steps

In [ ]:
print("""
╔═══════════════════════════════════════════════════════════════╗
║          ✅ TRAINING & EXPORT COMPLETED!                    ║
╚═══════════════════════════════════════════════════════════════╝

📋 What to do next:

1️⃣  EVALUATE MODEL
   → Open TEST_MODEL_COLAB.ipynb
   → Get accuracy, precision, recall, F1 scores
   → Visualize confusion matrix & predictions

2️⃣  DOWNLOAD MODELS
   → Google Drive: MyDrive/comparison_detector_models_v2/
   → Download best_model.h5 or final_model.keras
   → Download saved_model/ folder for FastAPI

3️⃣  DEPLOY LOCALLY
   → Copy models to d:\\ComparisonDetector\\trained_models\\
   → Run: python test_model_simple.py
   → Or: python test_model_advanced.py

4️⃣  IMPROVE MODEL
   → Check ROADMAP_CẢI_THIỆN_ĐẠT_MỨC_LÂM_SÀNG.md
   → Data augmentation
   → Fine-tune hyperparameters
   → Add regularization

📚 AVAILABLE GUIDES:
   - HƯỚNG_DẪN_SAU_KHI_TRAIN_COLAB.md (Post-training)
   - CẢI_THIỆN_ĐỘ_CHÍNH_XÁC.md (Accuracy improvements)
   - ROADMAP_CẢI_THIỆN_ĐẠT_MỨC_LÂM_SÀNG.md (Clinical-grade guide)

🎉 Good luck with your model!
""")